
# Deep Learning Approach at 



## Importing Code and Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [4]:
import os
import torch
import torch.nn as nn
import numpy as np
import pandas as pd 
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid
from torchvision import datasets
from torchvision import models
import nltk
import gensim
from gensim.models import Word2Vec


## Importing Data

data was located in a csv file. For this project, only the lyrics will be used to find a relation with the binary valence variable denoting if a song is happy or sad.

In [5]:
df = pd.read_csv('./drive/MyDrive/NLPproject/data_bin.csv')
df

,Unnamed: 0,Unnamed: 0.1,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre,lyrics,binary_valence
0,0,2949,https://www.last.fm/music/skywave/_/another%2b...,Another Love,Skywave,"['explosive', 'euphoric']",5,5.440000,4.582000,4.548000,3eba75ff-50d1-48eb-b39f-2e1e456c0847,NaN,shoegaze,"[Intro]\r\nIt's Rittz, bitch\r\n\r\n[Verse 1]\...",1
1,1,65958,https://www.last.fm/music/neneh%2bcherry/_/pea...,Peace In Mind,Neneh Cherry,['reflective'],4,4.841667,2.815000,4.950000,05011436-be05-433d-b7d1-405f14b45838,3qij053VLKWDprTlF6REDl,trip-hop,I've got a little piece of mind\r\nFrom the co...,0
2,2,43234,https://www.last.fm/music/adam%2bgreen/_/goblin,Goblin,Adam Green,['silly'],1,6.720000,4.860000,6.500000,93f50361-0581-4dd6-a294-93ca5bef904f,122tMrH7PcUWMN4AcklyU0,folk,When she stopped talking to me\r\nI knew that ...,1
3,3,5766,https://www.last.fm/music/ceremony/_/i%2bwant%...,I Want To Put This To An End,Ceremony,['intense'],2,3.995000,6.135000,4.585000,3b09dada-b877-4bda-b35a-34a1d28b1461,3HBwKXoQnPdi8Tzmioiwyv,hardcore,"My heart beats in slow songs, pumping moments ...",0
4,4,61410,https://www.last.fm/music/beyond%2bcreation/_/...,Coexistence,Beyond Creation,['technical'],2,5.693333,5.476275,6.390392,e4348ebc-97f2-4bcb-af2e-2998d0fae905,3TfnNa2rOCuGZi4IBWtYVE,technical death metal,When the universal darkness is covering the ea...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,5995,89595,https://www.last.fm/music/diane%2bbirch/_/noth...,Nothing But A Miracle,Diane Birch,['quiet'],6,6.062000,3.772000,5.639000,db1d9721-6dd4-40b5-89a2-42562e94086e,6IZVRuDF1R3vu7M7x6jAkM,soul,Gettin' tired of livin'\r\nLivin' for a moment...,1
5996,5996,25650,https://www.last.fm/music/tran%2bqual/_/montse...,Montserrat,Tran Qual,['relaxed'],9,5.326474,3.281561,5.053699,NaN,NaN,indie,None,1
5997,5997,28401,https://www.last.fm/music/raging%2bslab/_/here...,Here Lies,Raging Slab,"['sarcastic', 'sardonic', 'cynical']",4,2.742500,3.395000,3.935000,dc3b6de4-e085-4f36-a848-d42f4b405f36,NaN,rock,None,0
5998,5998,83488,https://www.last.fm/music/jurassic%2b5/_/back%...,Back 4 You,Jurassic 5,['confident'],2,7.386102,4.336271,6.921017,444318b7-ae48-4dde-a45c-ab3460faeb35,NaN,hip-hop,"[Intro: Soup]\r\nCheck this out, real quick\r\...",1


In [6]:
toDrop = df.index[df["lyrics"] == 'None'].tolist()
print(toDrop)
df.drop(toDrop, axis = 0, inplace = True)
print(df.index[df["lyrics"] =='None'].tolist())

[5, 6, 10, 11, 13, 14, 15, 17, 19, 20, 21, 25, 26, 31, 42, 47, 48, 50, 52, 61, 62, 68, 69, 74, 76, 85, 107, 108, 116, 117, 123, 129, 132, 133, 135, 136, 138, 141, 145, 148, 158, 161, 162, 163, 166, 167, 173, 180, 181, 186, 187, 189, 190, 198, 199, 200, 202, 203, 204, 206, 210, 212, 214, 215, 218, 219, 221, 222, 230, 232, 235, 236, 240, 241, 247, 249, 250, 254, 255, 258, 259, 261, 264, 268, 275, 277, 278, 279, 281, 285, 287, 290, 291, 293, 295, 297, 306, 309, 314, 330, 333, 334, 335, 336, 337, 343, 345, 347, 348, 351, 353, 355, 357, 358, 359, 361, 365, 366, 372, 374, 376, 377, 382, 385, 386, 388, 395, 402, 404, 405, 406, 407, 408, 410, 412, 413, 418, 422, 425, 428, 430, 440, 449, 451, 452, 456, 457, 459, 460, 462, 464, 466, 468, 475, 476, 477, 480, 483, 486, 487, 491, 495, 506, 516, 518, 520, 524, 526, 527, 530, 537, 539, 544, 546, 548, 549, 553, 555, 556, 560, 561, 562, 564, 566, 569, 570, 571, 572, 574, 575, 576, 579, 583, 584, 587, 591, 596, 598, 606, 609, 610, 611, 613, 614, 617, 62

In [100]:
y = df['binary_valence'].to_numpy()
X = df['lyrics'].to_numpy()


In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Tokenizing and Embeedding

the data was tokenized using nltk and then, using googles pretrained word2vec model, was converted into word embeddings to better be able to classify the data. 

In [10]:
delete_list = ['Intro','Verse','Hook']
X_tokenized = []
for lyrics in X:
  sent_tok = nltk.sent_tokenize(lyrics)
  word_tok = [nltk.word_tokenize(i) for i in sent_tok]
  word_tok = [word.lower() for sublist in word_tok for word in sublist if (word.isalpha() and word not in delete_list)]
  X_tokenized.append(word_tok)

In [11]:
bigmodel = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/NLPproject/GoogleNews-vectors-negative300-SLIM.bin.gz', binary=True)


In [12]:
song_vecs = []

for toks in X_tokenized:
  totvec = np.zeros(300)
  for tok in toks:
    if tok.lower() in bigmodel:
      totvec += bigmodel[tok.lower()]
  song_vecs.append(totvec)

print(len(song_vecs))
print(len(song_vecs[0]))

4111
300


## Getting Word Embedding Data Ready for Model

In order to have an efficient way of training and testing, the data was first shuffled and then divided in an 80-20 split (training-testing). Afterwards, the data was then supplied into a DataLoader object for proper interaction with the pytorch model later on.

In [15]:
indexes = np.array(range(len(y)))
np.random.shuffle(indexes)
indexes

array([1960, 2940,   69, ..., 2261, 4067,  722])

In [16]:
y_shuffled = []
X_shuffled = []
dataset = []
shuffled_dataset = []
for i in range(len(y)):
  dataset.append((torch.tensor(song_vecs[i]),y[i]))
for i in range(len(dataset)):
  shuffled_dataset.append(dataset[indexes[i]])

train_set = dataset[0:int(len(shuffled_dataset) * .8)]
test_set = dataset[0:int(len(shuffled_dataset) * .2)]

In [21]:
trainset = torch.utils.data.DataLoader(train_set,batch_size = 5, shuffle = True, num_workers = 2)
testset = torch.utils.data.DataLoader(test_set,batch_size = 5, shuffle = False, num_workers = 2)
classes = ('sad','happy')

## Feed Forward Neural Network Training and Fitting

For training this dataset a feed forward neural network was created. This network contains 5 layers, each being normalized by the relu function. The loss function used for this is cross entropy. Training, testing and recording of validation is constructed all in the class architecture.

For parameter initialization, the length of one input is used for input while the output is set to the class size, 2. Afterwards, the model is then sent to the device and  then further trained and tested. 

An average accuracy of 92% was achieved with a loss of 23%.

In [23]:
class Forward_NN(nn.Module):
  def __init__(self,input,output):
    super().__init__()
    self.layer1 = nn.Linear(input, input)
    self.layer2 = nn.Linear(input,input)
    self.layer3 = nn.Linear(input,input)
    self.layer4 = nn.Linear(input,100)
    self.layer5 = nn.Linear(100,output)
  
  def forward(self, input):
    out = self.layer1(input)
    out = F.relu(out)
    out = self.layer2(out)
    out = F.relu(out)
    out = self.layer3(out)
    out = F.relu(out) 
    out = self.layer4(out)
    out = F.relu(out)
    output = self.layer5(out)
    return output
  
  def training_step(self,batch):
    input,labels = batch
    input = input.to(device)
    labels = labels.to(device)
    out = self(input.float())
    loss = F.cross_entropy(out,labels)
    return loss
  
  def validation_step(self,batch):
    input, labels = batch
    input = input.to(device)
    labels = labels.to(device)
    out = self(input.float())
    loss = F.cross_entropy(out,labels)
    _, preds = torch.max(out, dim = 1)
    acc = torch.tensor(torch.sum(preds == labels).item() / len(preds))
    return {'loss':loss,'acc':acc}

  def epoch_val_end(self,outputs):
    batch_losses = [x['loss'] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()   
    batch_accs = [x['acc'] for x in outputs]
    epoch_acc = torch.stack(batch_accs).mean()      
    return {'loss': epoch_loss.item(), 'acc': epoch_acc.item()}

  def epoch_end(self, epoch, result):
    print("Epoch [{}], loss: {:.4f}, acc: {:.4f}".format(epoch, result['loss'], result['acc']))
  



In [99]:
fnn = Forward_NN(len(song_vecs[0]), 2)
device = torch.device('cuda')
fnn = fnn.to(device)

In [27]:
history = []
optimizer = torch.optim.SGD(fnn.parameters(),0.001)
for epoch in range(100):
  for batch in trainset:
    loss = fnn.training_step(batch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  outputs = [fnn.validation_step(batch) for batch in testset]
  results = fnn.epoch_val_end(outputs)
  fnn.epoch_end(epoch,results)
  history.append(results)

Epoch [0], loss: 0.4047, acc: 0.8170
Epoch [1], loss: 0.4764, acc: 0.7400
Epoch [2], loss: 0.4603, acc: 0.7418
Epoch [3], loss: 0.4043, acc: 0.8073
Epoch [4], loss: 0.4538, acc: 0.7618
Epoch [5], loss: 0.4256, acc: 0.7879
Epoch [6], loss: 0.4323, acc: 0.7939
Epoch [7], loss: 0.3906, acc: 0.8291
Epoch [8], loss: 0.4637, acc: 0.7697
Epoch [9], loss: 0.3817, acc: 0.8091
Epoch [10], loss: 0.3792, acc: 0.8448
Epoch [11], loss: 0.3976, acc: 0.8170
Epoch [12], loss: 0.3427, acc: 0.8545
Epoch [13], loss: 0.3855, acc: 0.8364
Epoch [14], loss: 0.3424, acc: 0.8679
Epoch [15], loss: 0.5635, acc: 0.7206
Epoch [16], loss: 0.3691, acc: 0.8509
Epoch [17], loss: 0.3507, acc: 0.8491
Epoch [18], loss: 0.4863, acc: 0.7673
Epoch [19], loss: 0.3467, acc: 0.8582
Epoch [20], loss: 0.3668, acc: 0.8279
Epoch [21], loss: 0.5061, acc: 0.7661
Epoch [22], loss: 0.3891, acc: 0.8267
Epoch [23], loss: 0.5951, acc: 0.6642
Epoch [24], loss: 0.3045, acc: 0.8655
Epoch [25], loss: 0.3216, acc: 0.8655
Epoch [26], loss: 0.43

In [36]:
feed_forward_df = pd.DataFrame(data = history)
accuracy = np.mean(feed_forward_df.acc[50:].to_numpy())
loss = np.mean(feed_forward_df.loss[50:].to_numpy())
print('For the last 50 inputs, accuracy was {:.4f} and loss was {:.4f}'.format(accuracy,loss))

For the last 50 inputs, accuracy was 0.9193 and loss was 0.2383


## Feed Forward Neural Network with Normalization Training and Fitting

In order to avoid overfitting the model onto the training data, the previously used feed forward architecture was changed. In this version, 4 layers are used and batch normalization and dropout are added to aid against normalized.

After this distinction is made, the same format is followed as above for training and sending the device to the GPU.

An accuracy of 68% was achieved with a loss of 60%

In [94]:
class fnn_normalized(nn.Module):
  def __init__(self,input,output):
    super().__init__()
    self.layer1 = nn.Linear(input, 64)
    self.layer2 = nn.Linear(64,64)
    self.layer3 = nn.Linear(64,50)
    self.layer4 = nn.Linear(50,output)
    self.dropout = nn.Dropout(p = 0.1)
    self.bn1 = nn.BatchNorm1d(64)
    self.bn2 = nn.BatchNorm1d(64)
    self.bn3 = nn.BatchNorm1d(50)
  
  def forward(self, input):
    out = self.layer1(input)
    out = F.relu(out)
    out = self.bn1(out)
    out = self.layer2(out)
    out = F.relu(out)
    out = self.bn2(out)
    out = self.layer3(out)
    out = F.relu(out)
    out = self.bn3(out)
    out = self.dropout(out)
    out = self.layer4(out)
    return out
  
  def training_step(self,batch):
    input,labels = batch
    input = input.to(device)
    labels = labels.to(device)
    out = self(input.float())
    loss = F.cross_entropy(out,labels)
    return loss
  
  def validation_step(self,batch):
    input, labels = batch
    input = input.to(device)
    labels = labels.to(device)
    out = self(input.float())
    loss = F.cross_entropy(out,labels)
    _, preds = torch.max(out, dim = 1)
    acc = torch.tensor(torch.sum(preds == labels).item() / len(preds))
    return {'loss':loss,'acc':acc}

  def epoch_val_end(self,outputs):
    batch_losses = [x['loss'] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()   
    batch_accs = [x['acc'] for x in outputs]
    epoch_acc = torch.stack(batch_accs).mean()      
    return {'loss': epoch_loss.item(), 'acc': epoch_acc.item()}

  def epoch_end(self, epoch, result):
    print("Epoch [{}], loss: {:.4f}, acc: {:.4f}".format(epoch, result['loss'], result['acc']))
  


In [95]:
fnn_n = fnn_normalized(len(song_vecs[0]), 2)
fnn_n = fnn_n.to(device)

In [97]:
history = []
optimizer = torch.optim.SGD(fnn_n.parameters(),0.001)
for epoch in range(100):
  for batch in trainset:
    loss = fnn_n.training_step(batch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  outputs = [fnn_n.validation_step(batch) for batch in testset]
  results = fnn_n.epoch_val_end(outputs)
  fnn_n.epoch_end(epoch,results)
  history.append(results)

Epoch [0], loss: 0.6047, acc: 0.6745
Epoch [1], loss: 0.6038, acc: 0.6867
Epoch [2], loss: 0.6022, acc: 0.6782
Epoch [3], loss: 0.6106, acc: 0.6818
Epoch [4], loss: 0.6197, acc: 0.6745
Epoch [5], loss: 0.6040, acc: 0.6879
Epoch [6], loss: 0.6160, acc: 0.6685
Epoch [7], loss: 0.6147, acc: 0.6685
Epoch [8], loss: 0.6179, acc: 0.6770
Epoch [9], loss: 0.6148, acc: 0.6915
Epoch [10], loss: 0.6081, acc: 0.6891
Epoch [11], loss: 0.6124, acc: 0.6891
Epoch [12], loss: 0.6127, acc: 0.6830
Epoch [13], loss: 0.6115, acc: 0.6721
Epoch [14], loss: 0.6045, acc: 0.6915
Epoch [15], loss: 0.6092, acc: 0.6855
Epoch [16], loss: 0.6040, acc: 0.6879
Epoch [17], loss: 0.6040, acc: 0.6927
Epoch [18], loss: 0.6035, acc: 0.6855
Epoch [19], loss: 0.6209, acc: 0.6673
Epoch [20], loss: 0.6172, acc: 0.6745
Epoch [21], loss: 0.6049, acc: 0.6939
Epoch [22], loss: 0.6087, acc: 0.6891
Epoch [23], loss: 0.6105, acc: 0.6758
Epoch [24], loss: 0.6105, acc: 0.6879
Epoch [25], loss: 0.6083, acc: 0.6952
Epoch [26], loss: 0.61

In [98]:
feed_forward_df = pd.DataFrame(data = history)
accuracy = np.mean(feed_forward_df.acc[50:].to_numpy())
loss = np.mean(feed_forward_df.loss[50:].to_numpy())
print('For the last 50 inputs, accuracy was {:.4f} and loss was {:.4f}'.format(accuracy,loss))

For the last 50 inputs, accuracy was 0.6832 and loss was 0.6017
